In [27]:
import sqlite3
import pandas as pd

testers=['S006','S008','S009','S010','S012','S013','S014','S015','S016','S017',\
         'S018','S019','S020','S021','S022','S023','S024','S025','S026','S027',\
            'S028','S029']

conn=sqlite3.connect('harth.db')

def dataOfTester(tester):
    return 'SELECT * FROM '+tester

def createDataframe(tester):
    tester_df=pd.read_sql_query(dataOfTester(tester),conn)
    if(tester=='S015' or tester=='S021'):
            tester_df=tester_df.drop(columns=['index'])
    if(tester=='S023'):
            tester_df=tester_df.drop(columns=['field1'])
    return tester_df

tester_df = [createDataframe(tester) for tester in testers]

tester_df[0]['timestamp']=pd.to_datetime(tester_df[0]['timestamp'])
interval=pd.Timedelta('0.010s')
tester_df[0]['sample']=(tester_df[0]['timestamp']-tester_df[0]['timestamp'].iloc[0])//interval
tester_df[0].drop(columns=['timestamp'],inplace=True)


# interval=pd.Timedelta('0.010s')
# df_nosens['sample']=(df_nosens['timestamp']-df_nosens['timestamp'].iloc[0])//interval

tester_df[0].head()



,back_x,back_y,back_z,thigh_x,thigh_y,thigh_z,label,sample
0,-0.760242,0.299570,0.468570,-5.092732,-0.298644,0.709439,6,0
1,-0.530138,0.281880,0.319987,0.900547,0.286944,0.340309,6,1
2,-1.170922,0.186353,-0.167010,-0.035442,-0.078423,-0.515212,6,2
3,-0.648772,0.016579,-0.054284,-1.554248,-0.950978,-0.221140,6,3
4,-0.355071,-0.051831,-0.113419,-0.547471,0.140903,-0.653782,6,4


In [101]:
rolling_window = tester_df[0].iloc[:,:-2].rolling(window=10,center=True).mean()
# rolling_window.dropna(inplace=True)
rolling_window['label']=tester_df[0]['label']
rolling_window.fillna(method='bfill',inplace=True,limit=5)
rolling_window.fillna(method='ffill',inplace=True,limit=5)
rolling_window.head(10)



row_47 = rolling_window.iloc[46]
print(row_47)
rolling_window.tail()


back_x    -0.986177
back_y     0.065087
back_z     0.086702
thigh_x   -0.961080
thigh_y    0.038694
thigh_z   -0.299277
label      1.000000
Name: 46, dtype: float64


C:\Users\gdste\AppData\Local\Temp\ipykernel_14352\2224458975.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  rolling_window.fillna(method='bfill',inplace=True,limit=5)
C:\Users\gdste\AppData\Local\Temp\ipykernel_14352\2224458975.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  rolling_window.fillna(method='ffill',inplace=True,limit=5)


,back_x,back_y,back_z,thigh_x,thigh_y,thigh_z,label
408704,-0.715584,0.03524,-0.693636,0.092638,0.070502,1.000754,7
408705,-0.715584,0.03524,-0.693636,0.092638,0.070502,1.000754,7
408706,-0.715584,0.03524,-0.693636,0.092638,0.070502,1.000754,7
408707,-0.715584,0.03524,-0.693636,0.092638,0.070502,1.000754,7
408708,-0.715584,0.03524,-0.693636,0.092638,0.070502,1.000754,7


In [102]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X=rolling_window.drop(columns=['label'])
Y=rolling_window['label']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [103]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=100,random_state=42,n_jobs=4)
rf.fit(X_train,y_train)

RandomForestClassifier(n_jobs=4, random_state=42)

In [104]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

y_pred=rf.predict(X_test)
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(rf.score(X_train,y_train))

0.9633970296787453
              precision    recall  f1-score   support

           1       0.85      0.91      0.88      7318
           3       0.84      0.71      0.77      3787
           4       0.93      0.66      0.77       434
           5       0.95      0.68      0.79       479
           6       0.96      0.97      0.97     18747
           7       1.00      1.00      1.00     76215
           8       1.00      1.00      1.00      3878
          13       0.82      0.95      0.88      7441
          14       0.79      0.55      0.65       910
         130       0.90      0.72      0.80      3404

    accuracy                           0.96    122613
   macro avg       0.90      0.81      0.85    122613
weighted avg       0.96      0.96      0.96    122613

1.0


In [105]:
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier(max_iter=1000,activation='tanh',random_state=42,early_stopping=True)
mlp.fit(X_train,y_train)


MLPClassifier(activation='tanh', early_stopping=True, max_iter=1000,
              random_state=42)

In [106]:
from sklearn.metrics import accuracy_score,classification_report
mlppredicts=mlp.predict(X_test)
mlp_report=classification_report(y_test,mlppredicts)
mlp_accur=accuracy_score(y_test,mlppredicts)
mlp_score=mlp.score(X_train,y_train)
print("MLP report:",mlp_report,"MLP accuracy:",mlp_accur,"MLP training:",mlp_score)

MLP report:               precision    recall  f1-score   support

           1       0.74      0.72      0.73      7318
           3       0.47      0.29      0.36      3787
           4       0.60      0.40      0.48       434
           5       0.82      0.52      0.64       479
           6       0.85      0.93      0.89     18747
           7       1.00      1.00      1.00     76215
           8       1.00      0.99      1.00      3878
          13       0.76      0.84      0.80      7441
          14       0.48      0.35      0.40       910
         130       0.67      0.65      0.66      3404

    accuracy                           0.92    122613
   macro avg       0.74      0.67      0.70    122613
weighted avg       0.91      0.92      0.92    122613
 MLP accuracy: 0.9200247934558325 MLP training: 0.919568955874951


In [107]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

GaussianNB()

In [108]:
predicts_gnb=gnb.predict(X_test)
gnb_report=classification_report(y_test,predicts_gnb)
gnb_accur=accuracy_score(y_test,predicts_gnb)
gnb_score=gnb.score(X_train,y_train)
print("GNB estim report: \n",gnb_report,'\n',"GNB estim accuracy:",gnb_accur,'\n',"GNB training:",gnb_score)

GNB estim report: 
               precision    recall  f1-score   support

           1       0.66      0.34      0.45      7318
           3       0.21      0.07      0.11      3787
           4       0.15      0.03      0.06       434
           5       0.73      0.13      0.22       479
           6       0.72      0.94      0.82     18747
           7       1.00      0.99      0.99     76215
           8       0.99      0.99      0.99      3878
          13       0.72      0.75      0.73      7441
          14       0.19      0.28      0.23       910
         130       0.52      0.65      0.58      3404

    accuracy                           0.88    122613
   macro avg       0.59      0.52      0.52    122613
weighted avg       0.87      0.88      0.87    122613
 
 GNB estim accuracy: 0.8799800999893975 
 GNB training: 0.8790965270398747
